# Final Project

## Author: Cho Laam Yuen

## Load Data and Packages

In [1]:
#connect to my google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#change working directory
%cd /content/drive/My\ Drive/Adv\ ML/gr5074-final-project-ChoLaamY/
!pwd

/content/drive/My Drive/Adv ML/gr5074-final-project-ChoLaamY
/content/drive/My Drive/Adv ML/gr5074-final-project-ChoLaamY


In [2]:
#load my libraries
import sys
import time
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os
import zipfile

from skimage.transform import resize
from sklearn.model_selection import train_test_split

from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.applications import VGG19, ResNet50, InceptionV3

In [6]:
#code taken from example code for Covid_Image_Classification (HW2)
#extracting all filenames iteratively
base_path = 'thumbnails'
categories = ['Automobile', 'Blog', 'Comedy', 'Entertainment', 'Food', 'Informative', 'News', 'Science', 'Tech', 'VideoGames']

# load file names to fnames list object
fnames = []
for category in categories:
    image_folder = os.path.join(base_path, category)
    file_names = os.listdir(image_folder)
    full_path = [os.path.join(image_folder, file_name) for file_name in file_names]
    fnames.append(full_path)

print('number of images for each category:', [len(f) for f in fnames])
print(fnames[0:2]) #examples of file names

number of images for each category: [261, 184, 168, 193, 268, 242, 193, 281, 260, 253]
[['thumbnails/Automobile/XUhXLsrZiE0.jpg', 'thumbnails/Automobile/9-41nLtleX0.jpg', 'thumbnails/Automobile/rKtFJxGzJX8.jpg', 'thumbnails/Automobile/PV1MlxtpIJs.jpg', 'thumbnails/Automobile/n30DARxk9A4.jpg', 'thumbnails/Automobile/8I-6_H0haAw.jpg', 'thumbnails/Automobile/6Lj_gFyI1Qs.jpg', 'thumbnails/Automobile/L_6SAsypMNM.jpg', 'thumbnails/Automobile/lf8daKt3UlY.jpg', 'thumbnails/Automobile/Quvby2Me5CM.jpg', 'thumbnails/Automobile/CFXfSBR5Q9w.jpg', 'thumbnails/Automobile/o690DovjDAc.jpg', 'thumbnails/Automobile/wZLvLDknpPs.jpg', 'thumbnails/Automobile/VpNX324owaQ.jpg', 'thumbnails/Automobile/_9qJ3rwwTDs.jpg', 'thumbnails/Automobile/kNZV3Bry5UU.jpg', 'thumbnails/Automobile/4-XDxCb92X4.jpg', 'thumbnails/Automobile/KTeXDOC_fRg.jpg', 'thumbnails/Automobile/daQcxVqQJsI.jpg', 'thumbnails/Automobile/DPthR7Z0um0.jpg', 'thumbnails/Automobile/iBy6XRHPODQ.jpg', 'thumbnails/Automobile/XRNv2yXqmaU.jpg', 'thumbnai

## Preprocess Data and Setup

In [ ]:
#writing the preprocessor
# Import image, load to array of shape height, width, channels, then min/max transform.
# Write preprocessor that will match up with model's expected input shape.
# Uses opencv for image preprocessing

def preprocessor(data, shape=(192, 192)):
        """
        This function reads in images, resizes them to a fixed shape, and
        min/max transforms them, before converting feature values to float32
        for ONNX.
        
        params:
            data
                list of unprocessed images
                      
        returns:
            X
                numpy array of preprocessed image data
                  
        """
           
        import cv2
        import numpy as np

        "Resize a color image and min/max transform the image"
        img = cv2.imread(data) # Read in image from filepath.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of blue green and red, we reverse the order for ML.
        #grayscale image?  Use im_gray = cv2.imread('gray_image.png', cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, shape) # Change height and width of image.
        img = img / 255.0 # Min-max transform.  

        # Resize the images.
        X = np.array(img)
        #X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels] if needed.
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        return X

#Try on single flower file (imports file and preprocesses it to data with following shape)
#preprocessor('COVID-19 Radiography Database/COVID/COVID (1).png').shape